# Filtro de Kalman

In [20]:
import numpy as np

In [34]:
def filtroDeKalman(posicao_anterior, deltaT, u_k, P):
    posicao_anterior = np.matrix(posicao_anterior).T
    x_kmenos1, y_kmenos1, theta_kmenos1 = posicao_anterior[0], posicao_anterior[1], posicao_anterior[2] 
    aceleracao_medida, velocidade_angular = u_k[0], u_k[1]
    deltaX = 1/2 * (deltaT ** 2) * aceleracao_medida[0] * np.cos(theta_kmenos1 + (deltaT*velocidade_angular))
    deltaY = 1/2 * (deltaT ** 2) * aceleracao_medida[1] * np.sin(theta_kmenos1 + (deltaT*velocidade_angular))
    deltaTheta = deltaT * velocidade_angular
    posicao_predita = posicao_anterior + np.matrix([[deltaX],
                                                    [deltaY],
                                                    [deltaTheta]])
    F_p = np.matrix([[1, 0, -1/2 * (deltaT ** 2) * aceleracao_medida[0] * np.sin(theta_kmenos1 + (deltaT*velocidade_angular))],
                     [0, 1, 1/2 * (deltaT ** 2) * aceleracao_medida[1] * np.cos(theta_kmenos1 + (deltaT*velocidade_angular))],
                     [0, 0, 1]])
    
    F_u = np.matrix([[1/2 * (deltaT ** 2) * np.cos(theta_kmenos1 + (deltaT*velocidade_angular)), 
                        -1/2 * (deltaT ** 3) * aceleracao_medida[0] * np.sin(theta_kmenos1 + (deltaT*velocidade_angular))],
                     [1/2 * (deltaT ** 2) * np.sin(theta_kmenos1 + (deltaT*velocidade_angular)),
                         1/2 * (deltaT ** 3) * aceleracao_medida[1] * np.cos(theta_kmenos1 + (deltaT*velocidade_angular))],
                     [0, deltaT]])
    
    Q = np.matrix([[1, 0],
                   [0, 1]])
    
    p_predito = F_p * P * F_p.T + F_u * Q * F_u.T
    
 


    print(p_predito)

In [ ]:
P = np.matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
filtroDeKalman([0, 1, 90], 100, [[0.1, 0.1], 0.3], P)

dF (x, y, theta) = {
    * 0 <= theta < pi/2:      |Yp - Sy| / |cos(theta)|
    * pi/2 <= theta < pi:     |Xp - Sx| / |sin(theta)|
    * pi <= theta < 3*pi/2:   Yp / |cos(theta)|
    * 3*pi/2 <= theta < 2*pi: Xp / |sin(theta)|  
}

dR (x, y, theta) = {
    * 0 <= theta < pi/2:      |Sx - Xp| / |cos(theta)
    * pi/2 <= theta < pi:     Yp / |sin(theta)|
    * pi <= theta < 3*pi/2:   Xp / |cos(theta)|
    * 3*pi/2 <= theta < 2*pi: |Sy - Yp| / |sin(theta)| 
}

dL (x, y, theta) = {
    * 0 <= theta < pi/2:      Xp / |cos(theta)
    * pi/2 <= theta < pi:     |Sy - Yp| / |sin(theta)|
    * pi <= theta < 3*pi/2:   |Sx - Xp| / |cos(theta)|
    * 3*pi/2 <= theta < 2*pi: Yp / |sin(theta)| 
}